In [1]:
import pandas as pd
import numpy as np
import json

C:\Users\Tanvi Yallapragada\AppData\Local\Temp\ipykernel_18276\1779657567.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Getting keys

In [ ]:
def analyze_json_structure(data, key_prefix=''):
    """
    Analyzes the structure of JSON data and identifies the main repeating components.

    Args:
    - data: JSON data (dictionary or list)
    - key_prefix: Prefix for the current key (used in recursive calls)

    Returns:
    - Dictionary containing information about the structure
    """
    structure = {}

    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{key_prefix}.{key}" if key_prefix else key
            if isinstance(value, (dict, list)):
                structure[new_key] = analyze_json_structure(value, new_key)
            else:
                structure[new_key] = type(value).__name__
    elif isinstance(data, list):
        if data:
            structure[key_prefix] = analyze_json_structure(data[0], key_prefix)
        else:
            structure[key_prefix] = 'Empty List'

    return structure

# Load JSON data from file
with open('annotations\\train2017.json', 'r') as f:
    json_data = json.load(f)

# Analyze the structure of the JSON data
json_structure = analyze_json_structure(json_data)

# Print the structure
print(json_structure)


#### Mapping the annotations with images

In [ ]:
import json
import cv2

# Load JSON data from file
with open('annotations/train2017.json', 'r') as f:
    data = json.load(f)

# Access images and annotations
images_data = data['images']
annotations_data = data['annotations']

# Load images and process annotations
for image_info in images_data:
    image_id = image_info['id']
    file_name = image_info['file_name']
    height = image_info['height']
    width = image_info['width']

    # Load image using OpenCV or PIL
    image = cv2.imread('JPEGImages/' + file_name)  # Adjust path as per your file structure

    print(f"Loaded image: {file_name}")
    print(f"Image ID: {image_id}")
    print(f"Image dimensions: {height} x {width}")

    # Process annotations for this image
    image_annotations = [annotation for annotation in annotations_data if annotation['image_id'] == image_id]
    print(f"Number of annotations for this image: {len(image_annotations)}")

    for annotation in image_annotations:
        category_id = annotation['category_id']
        segmentation = annotation['segmentation']
        area = annotation['area']
        bbox = annotation['bbox']
        is_crowd = annotation['iscrowd']

        print(f"Annotation - Category ID: {category_id}, Area: {area}, Bounding Box: {bbox}, Is Crowd: {is_crowd}")

    print("="*50)  # Separator between images

# Use image and annotations for your model training or evaluation


#### Seeing any 5 images

In [ ]:
# Display any 5 images and their corresponding annotations
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load JSON data from file
with open('annotations/train2017.json', 'r') as f:
    data = json.load(f)

# Access images and annotations
images_data = data['images']
annotations_data = data['annotations']

# Load and display images
num_images_to_display = 5
fig, axes = plt.subplots(1, num_images_to_display, figsize=(20, 5))

for i in range(num_images_to_display):
    # Load image using OpenCV or PIL
    image_info = images_data[i]
    file_name = image_info['file_name']
    image = cv2.imread('JPEGImages/' + file_name)  # Adjust path as per your file structure

    # Display image
    axes[i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f"Image ID: {image_info['id']}")
    axes[i].axis('off')

    # Process annotations for this image
    image_annotations = [annotation for annotation in annotations_data if annotation['image_id'] == image_info['id']]

    # Display annotations
    for annotation in image_annotations:
        bbox = annotation['bbox']
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='r', facecolor='none')
        axes[i].add_patch(rect)

plt.show()

#### Convert the json data to a dataframe

In [ ]:
df_images = pd.DataFrame(images_data)
df_annotations = pd.DataFrame(annotations_data)
print(df_images.head())

#### Process the test data

In [ ]:
import json
import cv2

# Load JSON data from the test file
with open('annotations/test2017.json', 'r') as f:
    test_data = json.load(f)

# Access images and annotations from the test data
test_images_data = test_data['images']
test_annotations_data = test_data['annotations']

# Load images and process annotations for the test data
for image_info in test_images_data:
    image_id = image_info['id']
    file_name = image_info['file_name']
    height = image_info['height']
    width = image_info['width']

    # Load the test image using OpenCV or PIL
    test_image = cv2.imread('JPEGImages/' + file_name)  # Adjust path as per your file structure

    print(f"Loaded test image: {file_name}")
    print(f"Test Image ID: {image_id}")
    print(f"Test Image dimensions: {height} x {width}")

    # Process annotations for the test image
    image_annotations = [annotation for annotation in test_annotations_data if annotation['image_id'] == image_id]
    print(f"Number of annotations for this test image: {len(image_annotations)}")

    for annotation in image_annotations:
        category_id = annotation['category_id']
        segmentation = annotation['segmentation']
        area = annotation['area']
        bbox = annotation['bbox']
        is_crowd = annotation['iscrowd']

        print(f"Annotation - Category ID: {category_id}, Area: {area}, Bounding Box: {bbox}, Is Crowd: {is_crowd}")

    print("=" * 50)  # Separator between test images

# Use test images and annotations for your model evaluation or testing

In [ ]:
df_images = pd.DataFrame(test_images_data)
df_annotations = pd.DataFrame(test_annotations_data)
print(df_images.head())

#### Convert annotations to yolo format (done on terminal)

#### Splitting into train and test based on annotations

In [ ]:
import os
import shutil

# Define input and output paths
image_folder = 'G:\\SAR\\HRSID_jpg\\JPEGImages'
train_annotation_folder = 'dataset\\labels\\train'
test_annotation_folder = 'dataset\\labels\\val'
output_train_folder = 'dataset\\images\\train'
output_val_folder = 'dataset\\images\\val'

# Iterate through image folder
for image_file in os.listdir(image_folder):
    if image_file.endswith('.jpg') or image_file.endswith('.jpeg') or image_file.endswith('.png'):
        image_name = os.path.splitext(image_file)[0]
        train_annotation_path = os.path.join(train_annotation_folder, f'{image_name}.txt')
        test_annotation_path = os.path.join(test_annotation_folder, f'{image_name}.txt')

        # Check if the image has corresponding annotations in train or test folders
        if os.path.exists(train_annotation_path):
            # Move image to the train folder
            shutil.copy(os.path.join(image_folder, image_file), os.path.join(output_train_folder, image_file))
        elif os.path.exists(test_annotation_path):
            # Move image to the test folder
            shutil.copy(os.path.join(image_folder, image_file), os.path.join(output_val_folder, image_file))
        else:
            print(f"No annotations found for {image_file}")


In [1]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='config.yaml', epochs=100, device='cuda', patience=10, seed=42, workers=2, dropout=0.2, batch=16)
path = model.export()

Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning C:\Users\hifia\Projects\Test\dataset\labels\train.cache... 486 images, 1027 backgrounds, 0 corrupt: 100%|██████████| 1513/1513 [00:00<?, ?it/s]
val: Scanning C:\Users\hifia\Projects\Test\dataset\labels\val.cache... 749 images, 454 backgrounds, 0 corrupt: 100%|██████████| 1203/1203 [00:00<?, ?it/s]


Plotting labels to runs\detect\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs\detect\train9
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.47G       1.61      5.146      1.072          4        640: 100%|██████████| 95/95 [00:15<00:00,  6.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:07<00:00,  4.87it/s]

                   all       1203       2120     0.0722      0.518      0.283      0.142


...

     87/100      2.46G     0.9977      1.475     0.9713         16        640: 100%|██████████| 95/95 [00:13<00:00,  7.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.51it/s]

                   all       1203       2120      0.529      0.664      0.491      0.321
Stopping training early as no improvement observed in last 10 epochs. Best results observed at epoch 77, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



87 epochs completed in 0.506 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.3MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:06<00:00,  5.87it/s]


                   all       1203       2120      0.548      0.656      0.496      0.339
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train9
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train9\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.2.0...
TorchScript: export success ✅ 1.2s, saved as 'runs\detect\train9\weights\best.torchscript' (11.9 MB)

Export complete (1.5s)
Results saved to C:\Users\hifia\Projects\Test\runs\detect\train9\weights
Predict:         yolo predict task=detect model=runs\detect\train9\weights\best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train9\weights\best.torchscript imgsz=640 data=config.yaml

Model 2

In [1]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='config2.yaml', epochs=100, device='cuda', patience=10, seed=42, workers=2, dropout=0.2, batch=16)
path = model.export()

Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=config2.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=2, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning C:\Users\hifia\Projects\Test\dataset2\labels\train... 4484 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4484/4484 [00:06<00:00, 700.14it/s]


train: New cache created: C:\Users\hifia\Projects\Test\dataset2\labels\train.cache


val: Scanning C:\Users\hifia\Projects\Test\dataset2\labels\val... 1120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1120/1120 [00:01<00:00, 575.17it/s]


val: New cache created: C:\Users\hifia\Projects\Test\dataset2\labels\val.cache
Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs\detect\train10
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     91/100      3.14G     0.8243     0.4568     0.8858          4        640: 100%|██████████| 281/281 [00:38<00:00,  7.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.93it/s]

                   all       1120       3603      0.908      0.826      0.916      0.655
Stopping training early as no improvement observed in last 10 epochs. Best results observed at epoch 81, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



91 epochs completed in 1.230 hours.
Optimizer stripped from runs\detect\train10\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train10\weights\best.pt, 6.3MB

Validating runs\detect\train10\weights\best.pt...
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.74it/s]


                   all       1120       3603      0.906       0.82      0.909      0.661
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train10
Ultralytics YOLOv8.1.11 🚀 Python-3.10.13 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train10\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.2.0...
TorchScript: export success ✅ 1.3s, saved as 'runs\detect\train10\weights\best.torchscript' (11.9 MB)

Export complete (1.6s)
Results saved to C:\Users\hifia\Projects\Test\runs\detect\train10\weights
Predict:         yolo predict task=detect model=runs\detect\train10\weights\best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train10\weights\best.torchscript imgsz=640 data=confi

Model 1 predictions

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("runs\detect\\train9\weights\\best.pt")  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['JPEGImages\\P0001_1200_2000_0_800.jpg', 'pure background\\P0001_3000_3800_10190_10990.png', 'JPEGImages\\P0001_3600_4400_5400_6200.jpg', 'pure background\\P0045_0_800_7200_8000.png'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Tanvi Yallapragada\AppData\Local\Temp\ipykernel_13520\3330521824.py:4: SyntaxWarning: invalid escape sequence '\d'
  model = YOLO("runs\detect\\train9\weights\\best.pt")  # pretrained YOLOv8n model



0: 640x640 1 ship, 1: 640x640 (no detections), 2: 640x640 1 ship, 3: 640x640 (no detections), 233.8ms
Speed: 3.3ms preprocess, 58.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


Model 2 predictions

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO("runs\detect\\train10\weights\\best.pt")  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['JPEGImages\\P0001_1200_2000_0_800.jpg', 'pure background\\P0001_3000_3800_10190_10990.png', 'JPEGImages\\P0001_3600_4400_5400_6200.jpg', 'pure background\\P0045_0_800_7200_8000.png'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Tanvi Yallapragada\AppData\Local\Temp\ipykernel_13520\447839559.py:4: SyntaxWarning: invalid escape sequence '\d'
  model = YOLO("runs\detect\\train10\weights\\best.pt")  # pretrained YOLOv8n model



0: 640x640 1 ship, 1: 640x640 (no detections), 2: 640x640 1 ship, 3: 640x640 (no detections), 230.6ms
Speed: 3.5ms preprocess, 57.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


In [2]:
model.summary()

NameError: name 'model' is not defined